In [1]:
import ROOT
from OutFunc import OutFunc

In [2]:
#Creating a rand column in monopole tree data, just to make OutFunc usable in its format
df = ROOT.RDataFrame("MonopoleTree", "../monopoleData.root")
df = df.Define("rand", "gRandom->Rndm()")
df.Snapshot("MonopoleTree", "inputMonopoleData.root")

<cppyy.gbl.ROOT.RDF.RResultPtr<ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager,void> > object at 0x573ac99c0c80>

In [3]:
# Set up the OutFunc object.  First argument must be one of the classifiers.
# 4th argument is offset for contour.
# 5th argument is bool array indicating which variables were used in training
dir = "dataset/weights/"
prefix = "tmvaTest"
name = "BDTcheck10"
tCut = 0.0
useVar = [False, True, True, True, True, True, True]

testStat = OutFunc(name, dir, prefix, tCut, useVar)

                         : Booking "BDTcheck10" of type "BDT" from dataset/weights//tmvaTest_BDTcheck10.weights.xml.
                         : Reading weight file: dataset/weights//tmvaTest_BDTcheck10.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
                         : Booked classifier "BDTcheck10" of type: "BDT"


In [4]:
#Filling histogram after evaluating model output for every entry.
inputFile = ROOT.TFile("inputMonopoleData.root")
tree = inputFile.Get("MonopoleTree")

hOutput = ROOT.TH1D("hOutput", "BDT, Output", 100, -1, 1)

numEntries = tree.GetEntries()
print(f"Evaluating {numEntries} events using BDT model:")

for i in range(numEntries):
    if i%1000000 == 0:
        print(f"{i} events evaluated!")    
    tree.GetEntry(i)
    t = testStat.val(tree)
    hOutput.Fill(t)

Evaluating 20948169 events using BDT model:
0 events evaluated!
1000000 events evaluated!
2000000 events evaluated!
3000000 events evaluated!
4000000 events evaluated!
5000000 events evaluated!
6000000 events evaluated!
7000000 events evaluated!
8000000 events evaluated!
9000000 events evaluated!
10000000 events evaluated!
11000000 events evaluated!
12000000 events evaluated!
13000000 events evaluated!
14000000 events evaluated!
15000000 events evaluated!
16000000 events evaluated!
17000000 events evaluated!
18000000 events evaluated!
19000000 events evaluated!
20000000 events evaluated!
                         : Rebuilding Dataset Default


In [5]:
#Saving output histogram
hFile = ROOT.TFile("outputMonopoleHisto.root", "RECREATE")
hOutput.Write()
hFile.Close()

In [6]:
#Plotting output histogram
hOutput.SetLineColor(ROOT.kBlue)
hOutput.SetLineWidth(2)
hOutput.SetFillColorAlpha(ROOT.kBlue, 0.3) 

%jsroot on

c = ROOT.TCanvas("c", "BDT Output", 800, 600)
ROOT.gStyle.SetOptStat(0)

hOutput.Draw("HIST")

#Legend
legend = ROOT.TLegend(0.45, 0.7, 0.65, 0.88)
legend.AddEntry(hOutput, "Monopole Data")

#Drawing and Saving canvas
c.Draw()
c.SaveAs("Monopole_output_curve.png")

Info in <TCanvas::Print>: png file Monopole_output_curve.png has been created


In [10]:
#The cut value decided using Punzi Method is 
cut = 0.3252525
#Thus we find the number of events in histogram above this to calculate number of events.
# Get the bin number corresponding to the threshold
start_bin = hOutput.FindBin(cut)

# Get the total number of bins
n_bins = hOutput.GetNbinsX()

# Compute the integral from start_bin to the last bin
integral_above = hOutput.Integral(start_bin, n_bins)

print(f"Integral of histogram above {cut} = {integral_above}")

Integral of histogram above 0.3252525 = 1.0


In [12]:
# Assume `h` is your existing TH1D histogram
x_min = 0.3252525 # <-- Set your lower bound
x_max = 1.0

# Step 1: Clone the original histogram
h_filtered = hOutput.Clone("h_filtered")
h_filtered.Reset()  # Clear bin contents, keep binning

# Step 2: Loop through bins and copy values in the range
for i in range(1, hOutput.GetNbinsX() + 1):
    bin_center = hOutput.GetBinCenter(i)
    if x_min <= bin_center <= x_max:
        h_filtered.SetBinContent(i, hOutput.GetBinContent(i))
        #h_filtered.SetBinError(i, hOutput.GetBinError(i))  # optional

# Step 3: Draw the filtered histogram
canvas = ROOT.TCanvas("c", "Filtered Histogram", 800, 600)
h_filtered.SetLineColor(ROOT.kBlue)
h_filtered.SetTitle(f"Histogram between {x_min} and {x_max}")
h_filtered.Draw()
canvas.Draw()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
